In [3]:
import os
import kagglehub
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame
from typing import  Optional
from scipy.stats import zscore

In [36]:
# Ensure the target directory exists
download_path = "../data/raw"
os.makedirs(download_path, exist_ok=True)

# Download the dataset
os.system(f'kaggle datasets download -d pavansubhasht/ibm-hr-analytics-attrition-dataset -p {download_path}')

# Unzip the downloaded file
with zipfile.ZipFile(f"{download_path}/ibm-hr-analytics-attrition-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall(download_path)

print(f"Dataset files are extracted to: {download_path}")

Dataset URL: https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset
License(s): DbCL-1.0



100%|██████████| 50.1k/50.1k [00:00<00:00, 269kB/s]


Dataset files are extracted to: ../data/raw


## Data preparation

In [31]:
df = pd.read_csv("../data/raw/WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [32]:
df.shape

(1470, 35)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [7]:
df.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,1470.0,36.923810,9.135373,18.0,30.00,36.0,43.00,60.0
DailyRate,1470.0,802.485714,403.509100,102.0,465.00,802.0,1157.00,1499.0
DistanceFromHome,1470.0,9.192517,8.106864,1.0,2.00,7.0,14.00,29.0
Education,1470.0,2.912925,1.024165,1.0,2.00,3.0,4.00,5.0
EmployeeCount,1470.0,1.000000,0.000000,1.0,1.00,1.0,1.00,1.0
EmployeeNumber,1470.0,1024.865306,602.024335,1.0,491.25,1020.5,1555.75,2068.0
EnvironmentSatisfaction,1470.0,2.721769,1.093082,1.0,2.00,3.0,4.00,4.0
HourlyRate,1470.0,65.891156,20.329428,30.0,48.00,66.0,83.75,100.0
JobInvolvement,1470.0,2.729932,0.711561,1.0,2.00,3.0,3.00,4.0
JobLevel,1470.0,2.063946,1.106940,1.0,1.00,2.0,3.00,5.0


In [9]:
def standardize_column_names(df: DataFrame) -> DataFrame:
    '''standardizes the columns name
      Args:
      df: DataFrame
      Returns:
        DataFrame
    '''
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    return df

In [10]:
df = standardize_column_names(df)
df.head().T

,0,1,2,3,4
age,41,49,37,33,27
attrition,Yes,No,Yes,No,No
businesstravel,Travel_Rarely,Travel_Frequently,Travel_Rarely,Travel_Frequently,Travel_Rarely
dailyrate,1102,279,1373,1392,591
department,Sales,Research & Development,Research & Development,Research & Development,Research & Development
distancefromhome,1,8,2,3,2
education,2,1,2,4,1
educationfield,Life Sciences,Life Sciences,Other,Life Sciences,Medical
employeecount,1,1,1,1,1
employeenumber,1,2,4,5,7


In [11]:
def clean_categorical_columns(df:DataFrame) -> DataFrame:
    '''clean the columns removing spaces, special characters and converting to lower case
    Args:
        df: DataFrame
    Returns:
        DataFrame
    '''
    clean_categorical_columns = df.select_dtypes(include=['object']).columns
    for column in clean_categorical_columns:
        df[column] = df[column].str.lower().str.strip().str.replace(' ', '_').str.replace('/', '_').str.replace('-', '_').str.replace('(', '').str.replace(')', '')
    return df

In [12]:
clean_categorical_columns(df)
df.head()

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,...,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager
0,41,yes,travel_rarely,1102,sales,1,2,life_sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,no,travel_frequently,279,research_&_development,8,1,life_sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,yes,travel_rarely,1373,research_&_development,2,2,other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,no,travel_frequently,1392,research_&_development,3,4,life_sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,no,travel_rarely,591,research_&_development,2,1,medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [13]:
df.duplicated().sum()

0

In [14]:
df.education.value_counts()

education
3    572
4    398
2    282
1    170
5     48
Name: count, dtype: int64

In [15]:
df.performancerating.value_counts()

performancerating
3    1244
4     226
Name: count, dtype: int64

In [16]:
performanceRating_map = {1: 'low', 2: 'good', 3: 'excellent', 4: 'outstanding'}
df['performancerating'] = df['performancerating'].map(performanceRating_map).astype('category')

In [17]:
df.performancerating.value_counts() 

performancerating
excellent      1244
outstanding     226
Name: count, dtype: int64

In [18]:
df.environmentsatisfaction.value_counts()

environmentsatisfaction
3    453
4    446
2    287
1    284
Name: count, dtype: int64

In [19]:
enviromentalstaisfaction_map = {1: 'low', 2: 'medium', 3: 'high', 4: 'very_high'}
df['environmentsatisfaction'] = df['environmentsatisfaction'].map(enviromentalstaisfaction_map).astype('category')

In [20]:
df.environmentsatisfaction.value_counts()

environmentsatisfaction
high         453
very_high    446
medium       287
low          284
Name: count, dtype: int64

In [21]:
df.jobinvolvement.value_counts()  

jobinvolvement
3    868
2    375
4    144
1     83
Name: count, dtype: int64

In [22]:
jobInvolvement_map = {1: 'low', 2: 'medium', 3: 'high', 4: 'very_high'}
df['jobinvolvement'] = df['jobinvolvement'].map(jobInvolvement_map).astype('category')

In [30]:
df.jobinvolvement.value_counts()

Series([], Name: count, dtype: int64)

In [28]:
jobInvolvement_map = {1: 'low', 2: 'medium', 3: 'high', 4: 'very_high'}
df['jobinvolvement'] = df['jobinvolvement'].map(jobInvolvement_map).astype('category')

In [29]:
df.jobinvolvement.value_counts()

Series([], Name: count, dtype: int64)